### 研究思路及分析过程
* 任务一：数据预处理
  * 缺失值分析
    * 数值为0/空值的的情况需要分开讨论，且关注缺失数据是否为真实缺失
  * 异常情况分析
    * 对出现'--'的情况进行分析，且关注该符号的实际意义以及占比情况
  * 重复情况分析
    - 对于重复数据进行删除
- 任务二：用户整体情况分析
  - 用户分布分析
    - 根据海内外，省份分析，乡镇分析入手，找到核心差异点所在
  - 用户活跃分析
    - 细分整体情况与工作日差异
  - 用户流失情况分析
    - 细分整体情况与用户流失风险
  - 线上管理决策建议
    - 宣传，活跃度，流失为切口进行分析
- 任务三：用户课程选择分析
  - 用户参与课程情况
    - 现有课程选择分析与受欢迎度计算
  - 用户课程推荐----基于协同过滤算法
    - 基于协同过滤算法进行重点课程推荐
  - 收费课程与用户学习进度相关分析
  - 线上课程综合推荐策略制定

In [1]:
pip install chinese_calendar

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for chinese-calendar, since package 'wheel' is not installed.
    Running setup.py install for chinese-calendar: started
    Running setup.py install for chinese-calendar: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\hommet74\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import datetime
import jieba
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['font.sans-serif']=['SimHei']
matplotlib.rcParams['font.family']='sans-serif'
matplotlib.rcParams['axes.unicode_minus']=False
from chinese_calendar import is_workday
from pyecharts.charts import Bar

### 1.1缺失值处理
首先判断该缺失值是否为真实缺失，针对不同的数据缺失情况，本次分析将会采用不同的处理方式：
- 1. 针对数值为0的情况，需要进行实际的分析，回归到原始数据中去，判断该数据为0时是否具有实际的意义，如果没有将其作为缺失值删除处理
- 2. 针对数据为空值的情况，如果该特征数据缺失情况低于10%，则结合该特征的重要性进行综合判断，如果字段重要性较低，则考虑直接删除，如果字段重要性较高，则直接插值法或者采用数据均值进行填补。

In [17]:
login=pd.read_csv('./教育平台的线上课程用户行为数据集/login.csv',encoding='gbk')
login.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387144 entries, 0 to 387143
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      387144 non-null  object
 1   login_time   387144 non-null  object
 2   login_place  387144 non-null  object
dtypes: object(3)
memory usage: 8.9+ MB


In [18]:
login.isnull().sum()

user_id        0
login_time     0
login_place    0
dtype: int64

### 1.2重复值处理
一天内重复登陆的行为视为一次登录即可

In [22]:
login['login_time']=pd.to_datetime(login['login_time'])
login['login_time']=login['login_time'].apply(lambda x:x.strftime('%Y-%m-%d'))  
login.drop_duplicates(inplace=True)
login

,user_id,login_time,login_place
0,用户3,2018-09-06,中国广东广州
1,用户3,2018-09-07,中国广东广州
5,用户3,2018-09-10,中国广东广州
8,用户3,2018-09-10,中国北京
10,用户3,2018-09-10,中国广东
...,...,...,...
387139,用户44247,2020-06-18,中国湖北武汉
387140,用户44247,2020-06-18,中国湖北
387141,用户44248,2020-06-18,中国天津
387142,用户44249,2020-06-18,中国北京


In [25]:
login['时间差1']=pd.to_datetime(login['login_time']).max()-pd.to_datetime(login['login_time'])
login=login.reset_index()
del login['index']

In [26]:
login

,user_id,login_time,login_place,时间差1
0,用户3,2018-09-06,中国广东广州,651 days
1,用户3,2018-09-07,中国广东广州,650 days
2,用户3,2018-09-10,中国广东广州,647 days
3,用户3,2018-09-10,中国北京,647 days
4,用户3,2018-09-10,中国广东,647 days
...,...,...,...,...
267720,用户44247,2020-06-18,中国湖北武汉,0 days
267721,用户44247,2020-06-18,中国湖北,0 days
267722,用户44248,2020-06-18,中国天津,0 days
267723,用户44249,2020-06-18,中国北京,0 days


In [27]:
#地区拆分
for i in range(login.shape[0]):
    if login.loc[i,'login_place'][0:2]=='中国':
        login.loc[i,'国家']='中国'
        if '黑龙江' in login.loc[i,'login_place']:
            login.loc[i,'省份']='黑龙江'
            if len(login.loc[i,'login_place'])>5:
                login.loc[i,'地区']=login.loc[i,'login_place'][5:]
            else:pass
        if '新疆维吾尔' in login.loc[i,'login_place']:
            login.loc[i,'省份']='新疆维吾尔'
            if len(login.loc[i,'login_place'])>7:
                login.loc[i,'地区']=login.loc[i,'login_place'][7:]
            else:pass
        if '内蒙古' in login.loc[i,'login_place']:
            login.loc[i,'省份']='内蒙古'
            if len(login.loc[i,'login_place'])>5:
                login.loc[i,'地区']=login.loc[i,'login_place'][5:]
            else:pass
        else:
            login.loc[i,'省份']=login.loc[i,'login_place'][2:4]
            login.loc[i,'地区']=login.loc[i,'login_place'][4:]
    else:
        li=[word for word in jieba.cut(login.iloc[i,2])]
        if len(li)==2:
            login.loc[i,'国家']=li[0]
            login.loc[i,'省份']=li[1]
        else:
            login.loc[i,'国家']=li[0]
    if i%10000==0:
        print(f'{round(i*100/(int(login.shape[0])),2)}%')

0.0%


Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\hommet74\AppData\Local\Temp\jieba.cache
Loading model cost 0.446 seconds.
Prefix dict has been built successfully.


3.74%
7.47%
11.21%
14.94%
18.68%
22.41%
26.15%


KeyboardInterrupt: 

In [30]:
login=pd.read_excel('./教育平台的线上课程用户行为数据集/用户地区切割.xlsx',index_col=0)
login

,user_id,login_time,login_place,时间差1,国家,省份,地区
0,用户3,2018-09-06,中国广东广州,651,中国,广东,广州
1,用户3,2018-09-07,中国广东广州,650,中国,广东,广州
2,用户3,2018-09-10,中国广东广州,647,中国,广东,广州
3,用户3,2018-09-10,中国北京,647,中国,北京,NaN
4,用户3,2018-09-10,中国广东,647,中国,广东,NaN
...,...,...,...,...,...,...,...
267720,用户44247,2020-06-18,中国湖北武汉,0,中国,湖北,武汉
267721,用户44247,2020-06-18,中国湖北,0,中国,湖北,NaN
267722,用户44248,2020-06-18,中国天津,0,中国,天津,NaN
267723,用户44249,2020-06-18,中国北京,0,中国,北京,NaN


- 至此已经提取出login表中所有可能有用的信息

In [31]:
stu_info=pd.read_csv('./教育平台的线上课程用户行为数据集/study_information.csv',encoding='gbk')
stu_info

,user_id,course_id,course_join_time,learn_process,price
0,用户3,课程106,2020-04-21 10:11:50,width: 0%;,0.0
1,用户3,课程136,2020-03-05 11:44:36,width: 1%;,0.0
2,用户3,课程205,2018-09-10 18:17:01,width: 63%;,0.0
3,用户4,课程26,2020-03-31 10:52:51,width: 0%;,319.0
4,用户4,课程34,2020-03-31 10:52:49,width: 0%;,299.0
...,...,...,...,...,...
194969,用户44245,课程76,2020-06-17 22:16:56,width: 0%;,0.0
194970,用户44246,课程76,2020-06-17 22:36:41,width: 4%;,0.0
194971,用户44247,课程19,2020-06-18 07:54:07,width: 100%;,499.0
194972,用户44247,课程32,2020-06-18 07:41:38,width: 0%;,499.0


In [32]:
stu_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194974 entries, 0 to 194973
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           194974 non-null  object 
 1   course_id         194974 non-null  object 
 2   course_join_time  194974 non-null  object 
 3   learn_process     194974 non-null  object 
 4   price             190736 non-null  float64
dtypes: float64(1), object(4)
memory usage: 7.4+ MB


In [34]:
stu_info.drop_duplicates(inplace=True)
stu_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194974 entries, 0 to 194973
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           194974 non-null  object 
 1   course_id         194974 non-null  object 
 2   course_join_time  194974 non-null  object 
 3   learn_process     194974 non-null  object 
 4   price             190736 non-null  float64
dtypes: float64(1), object(4)
memory usage: 8.9+ MB


In [35]:
stu_info[stu_info.price.isnull()].course_id.value_counts()

课程51    4011
课程96     227
Name: course_id, dtype: int64

- 查看是否有差异定价

In [36]:
stu_course=stu_info.groupby('course_id').agg({'price':['max','min']})
stu_course[(stu_course['price']['max']-stu_course['price']['min'])!=0]

price    
            max min
course_id          
课程51        NaN NaN
课程96        NaN NaN

In [37]:
#将进度转为数值型,从而便于计算
stu_info['learn_process']=stu_info['learn_process'].apply(lambda x:int(x.split('width:')[1].split('%')[0]))
stu_info['course_join_time']=pd.to_datetime(stu_info['course_join_time'])
stu_info['course_join_time']=stu_info['course_join_time'].apply(lambda x: x.strftime('%Y-%m-%d'))
stu_info

,user_id,course_id,course_join_time,learn_process,price
0,用户3,课程106,2020-04-21,0,0.0
1,用户3,课程136,2020-03-05,1,0.0
2,用户3,课程205,2018-09-10,63,0.0
3,用户4,课程26,2020-03-31,0,319.0
4,用户4,课程34,2020-03-31,0,299.0
...,...,...,...,...,...
194969,用户44245,课程76,2020-06-17,0,0.0
194970,用户44246,课程76,2020-06-17,4,0.0
194971,用户44247,课程19,2020-06-18,100,499.0
194972,用户44247,课程32,2020-06-18,0,499.0


- 至此已经提取出stu表中所有可能有用的信息

In [76]:
users=pd.read_csv('./教育平台的线上课程用户行为数据集/users.csv',encoding='gbk')
users

,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school
0,用户44251,2020/6/18 9:49,2020/6/18 9:49,0,0,41.25,NaN
1,用户44250,2020/6/18 9:47,2020/6/18 9:48,0,0,0.00,NaN
2,用户44249,2020/6/18 9:43,2020/6/18 9:43,0,0,16.22,NaN
3,用户44248,2020/6/18 9:09,2020/6/18 9:09,0,0,0.00,NaN
4,用户44247,2020/6/18 7:41,2020/6/18 8:15,0,0,1.80,NaN
...,...,...,...,...,...,...,...
43978,用户6,2018/9/11 16:13,2018/9/11 16:14,0,0,0.00,NaN
43979,用户5,2018/9/10 15:48,2020/6/15 17:13,9,5,2116.15,NaN
43980,用户4,2018/9/10 14:15,2020/6/5 9:50,4,0,341.20,NaN
43981,用户3,2018/9/4 13:32,2020/6/18 9:18,2,1,370.35,NaN


In [41]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43983 entries, 0 to 43982
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   user_id                 43916 non-null  object 
 1   register_time           43983 non-null  object 
 2   recently_logged         43983 non-null  object 
 3   number_of_classes_join  43983 non-null  int64  
 4   number_of_classes_out   43983 non-null  int64  
 5   learn_time              43983 non-null  float64
 6   school                  10571 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 2.3+ MB


In [77]:
users=users[~users.user_id.isnull()]
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43916 entries, 0 to 43982
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   user_id                 43916 non-null  object 
 1   register_time           43916 non-null  object 
 2   recently_logged         43916 non-null  object 
 3   number_of_classes_join  43916 non-null  int64  
 4   number_of_classes_out   43916 non-null  int64  
 5   learn_time              43916 non-null  float64
 6   school                  10569 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 2.7+ MB


In [78]:
users['是否填写学校信息']=users['school']
users.是否填写学校信息[~users['是否填写学校信息'].isnull()]=1
users.是否填写学校信息[users['是否填写学校信息'].isnull()]=0
users.是否填写学校信息.value_counts()

C:\Users\hommet74\AppData\Local\Temp/ipykernel_7392/2388223601.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users['是否填写学校信息']=users['school']
C:\Users\hommet74\AppData\Local\Temp/ipykernel_7392/2388223601.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users.是否填写学校信息[~users['是否填写学校信息'].isnull()]=1
C:\Users\hommet74\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

0    33347
1    10569
Name: 是否填写学校信息, dtype: int64

### 1.3异常值处理
- 可以看出有一些recently_logged时间和现在的时间很接近，有一些很远，因此可以将'--'进行进一步分析
  - 用户注册后未登录
  - 用户注册后就未退出登录 使用login中的最新登录信息进行替换
 
因此考虑使用学习时间加上注册的时间作为最近登录时间，且设置一天学习8小时为上限

In [79]:
keys=login.groupby('user_id').login_time.max().index.tolist()
values=login.groupby('user_id').login_time.max().values.tolist()

In [80]:
login_time={}
for i in range(len(keys)):
    login_time[keys[i]]=values[i]
u_2=users[users.recently_logged!='--']
u_1=users[users.recently_logged=='--']


In [81]:
for i in range(u_1.shape[0]):
    if u_1.iloc[i,0] in login_time.keys():
        u_1.iloc[i,2]=pd.to_datetime(login_time[u_1.iloc[i,0]])
    else:
        if pd.to_datetime(u_1.iloc[i,1])+datetime.timedelta(days  = int(u_1.iloc[i,5])/480 )>pd.to_datetime('2020-06-18'):
            u_1.iloc[i,2]=pd.to_datetime('2020-06-18')
            print('修改时间为最新时间')
        else:
            u_1.iloc[i,2]=pd.to_datetime(u_1.iloc[i,1])+datetime.timedelta(days  = int(u_1.iloc[i,5])/480 )        
users=pd.concat([u_1,u_2])

C:\Users\hommet74\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


修改时间为最新时间
修改时间为最新时间
修改时间为最新时间
修改时间为最新时间
修改时间为最新时间
修改时间为最新时间
修改时间为最新时间
修改时间为最新时间


In [82]:
users

,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,是否填写学校信息
11,用户44240,2020/6/17 17:25,2020-06-18 00:00:00,1,0,1667.28,NaN,0
12,用户44239,2020/6/17 17:24,2020-06-18 00:00:00,1,0,2109.75,NaN,0
14,用户44235,2020/6/17 16:39,2020-06-17 16:39:00,1,0,0.00,NaN,0
15,用户44237,2020/6/17 16:39,2020-06-18 00:00:00,1,0,10348.62,NaN,0
16,用户44232,2020/6/17 16:39,2020-06-18 00:00:00,1,0,9054.72,NaN,0
...,...,...,...,...,...,...,...,...
43978,用户6,2018/9/11 16:13,2018/9/11 16:14,0,0,0.00,NaN,0
43979,用户5,2018/9/10 15:48,2020/6/15 17:13,9,5,2116.15,NaN,0
43980,用户4,2018/9/10 14:15,2020/6/5 9:50,4,0,341.20,NaN,0
43981,用户3,2018/9/4 13:32,2020/6/18 9:18,2,1,370.35,NaN,0


In [53]:
#以最新的时间为基准，计算出用户登录注册行为的时间差


11     2020-06-18 00:00:00
12     2020-06-18 00:00:00
14     2020-06-17 16:39:00
15     2020-06-18 00:00:00
16     2020-06-18 00:00:00
              ...         
311    2020-06-11 18:20:00
312    2020-06-11 18:20:00
313    2020-06-11 18:20:00
314    2020-06-11 18:20:00
315    2020-06-11 18:20:00
Name: recently_logged, Length: 100, dtype: object

In [83]:
users.register_time=pd.to_datetime(users.register_time).apply(lambda x:x.strftime('%Y-%m-%d'))
users.recently_logged=pd.to_datetime(users.recently_logged).apply(lambda x:x.strftime('%Y-%m-%d'))

In [84]:
users['register_logged_time']=pd.to_datetime(users['recently_logged'])-pd.to_datetime(users['register_time'])
users['register_now_time']=pd.to_datetime(users['register_time']).max()-pd.to_datetime(users['register_time'])
users['logged_now_time']=pd.to_datetime(users['recently_logged']).max()-pd.to_datetime(users['register_time'])
users

,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,是否填写学校信息,register_logged_time,register_now_time,logged_now_time
11,用户44240,2020-06-17,2020-06-18,1,0,1667.28,NaN,0,1 days,1 days,1 days
12,用户44239,2020-06-17,2020-06-18,1,0,2109.75,NaN,0,1 days,1 days,1 days
14,用户44235,2020-06-17,2020-06-17,1,0,0.00,NaN,0,0 days,1 days,1 days
15,用户44237,2020-06-17,2020-06-18,1,0,10348.62,NaN,0,1 days,1 days,1 days
16,用户44232,2020-06-17,2020-06-18,1,0,9054.72,NaN,0,1 days,1 days,1 days
...,...,...,...,...,...,...,...,...,...,...,...
43978,用户6,2018-09-11,2018-09-11,0,0,0.00,NaN,0,0 days,646 days,646 days
43979,用户5,2018-09-10,2020-06-15,9,5,2116.15,NaN,0,644 days,647 days,647 days
43980,用户4,2018-09-10,2020-06-05,4,0,341.20,NaN,0,634 days,647 days,647 days
43981,用户3,2018-09-04,2020-06-18,2,1,370.35,NaN,0,653 days,653 days,653 days


In [85]:
#现在仍加入的班级数量
users['number_of_classes_now']=users['number_of_classes_join']-users['number_of_classes_out']
users

,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,是否填写学校信息,register_logged_time,register_now_time,logged_now_time,number_of_classes_now
11,用户44240,2020-06-17,2020-06-18,1,0,1667.28,NaN,0,1 days,1 days,1 days,1
12,用户44239,2020-06-17,2020-06-18,1,0,2109.75,NaN,0,1 days,1 days,1 days,1
14,用户44235,2020-06-17,2020-06-17,1,0,0.00,NaN,0,0 days,1 days,1 days,1
15,用户44237,2020-06-17,2020-06-18,1,0,10348.62,NaN,0,1 days,1 days,1 days,1
16,用户44232,2020-06-17,2020-06-18,1,0,9054.72,NaN,0,1 days,1 days,1 days,1
...,...,...,...,...,...,...,...,...,...,...,...,...
43978,用户6,2018-09-11,2018-09-11,0,0,0.00,NaN,0,0 days,646 days,646 days,0
43979,用户5,2018-09-10,2020-06-15,9,5,2116.15,NaN,0,644 days,647 days,647 days,4
43980,用户4,2018-09-10,2020-06-05,4,0,341.20,NaN,0,634 days,647 days,647 days,4
43981,用户3,2018-09-04,2020-06-18,2,1,370.35,NaN,0,653 days,653 days,653 days,1


- 至此已经完成所有的信息提取与数据预处理过程

In [57]:
stu_info

,user_id,course_id,course_join_time,learn_process,price
0,用户3,课程106,2020-04-21,0,0.0
1,用户3,课程136,2020-03-05,1,0.0
2,用户3,课程205,2018-09-10,63,0.0
3,用户4,课程26,2020-03-31,0,319.0
4,用户4,课程34,2020-03-31,0,299.0
...,...,...,...,...,...
194969,用户44245,课程76,2020-06-17,0,0.0
194970,用户44246,课程76,2020-06-17,4,0.0
194971,用户44247,课程19,2020-06-18,100,499.0
194972,用户44247,课程32,2020-06-18,0,499.0


In [63]:
#计算选课数量
stu_count=stu_info.groupby('user_id').agg(选课数量=('course_id','count'))
stu_count

,选课数量
user_id,
用户10,2
用户100,1
用户10000,1
用户10001,1
用户10002,1
...,...
用户9993,1
用户9994,1
用户9995,1


In [86]:
def nx_data(df=stu_info,group_name=['course_id','user_id']):
    # 得到共现字典
    user_dic={}  
    stu_info_data=df.groupby(group_name)['course_id'].count().unstack()
    column=stu_info_data.columns.tolist()
    for i in range(stu_info_data.shape[0]):
        user_dic[column[i]]=stu_info_data[stu_info_data[column[i]]==1].index.tolist()
        
    #构造共现矩阵
    course_name=list(set(df['course_id'].values.tolist()))
    course_data=pd.DataFrame(data=np.zeros(shape=(len(course_name),len(course_name))),index=course_name,columns=course_name)
    for value in user_dic.values():
        if len(value)==1:
            pass
        else:
            for i in range(len(value)):
                for j in range(i+1,len(value)):
                    course_data.loc[value[i],value[j]]+=1
    return (user_dic,course_data)

user_dic,course_data=nx_data()

for i,key in enumerate(user_dic.keys()):
    users.loc[i,'选课数量']=len(user_dic[key])
users


,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,是否填写学校信息,register_logged_time,register_now_time,logged_now_time,number_of_classes_now,选课数量
11,用户44240,2020-06-17,2020-06-18,1,0,1667.28,NaN,0,1 days,1 days,1 days,1,1.0
12,用户44239,2020-06-17,2020-06-18,1,0,2109.75,NaN,0,1 days,1 days,1 days,1,1.0
14,用户44235,2020-06-17,2020-06-17,1,0,0.00,NaN,0,0 days,1 days,1 days,1,1.0
15,用户44237,2020-06-17,2020-06-18,1,0,10348.62,NaN,0,1 days,1 days,1 days,1,4.0
16,用户44232,2020-06-17,2020-06-18,1,0,9054.72,NaN,0,1 days,1 days,1 days,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43978,用户6,2018-09-11,2018-09-11,0,0,0.00,NaN,0,0 days,646 days,646 days,0,NaN
43979,用户5,2018-09-10,2020-06-15,9,5,2116.15,NaN,0,644 days,647 days,647 days,4,NaN
43980,用户4,2018-09-10,2020-06-05,4,0,341.20,NaN,0,634 days,647 days,647 days,4,NaN
43981,用户3,2018-09-04,2020-06-18,2,1,370.35,NaN,0,653 days,653 days,653 days,1,NaN


### 地区合并
取最近的登录地点合并

In [98]:
login_1=login.sort_values(by=['user_id','时间差1'])
login_del=login_1.user_id.drop_duplicates()
login_diff=login.iloc[list(login_del.index),:]
users_all=pd.merge(users,login_diff,how='left')
users_all=users_all.reset_index()
users_all=users_all.drop(columns=['index'])
users_all

,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,是否填写学校信息,register_logged_time,register_now_time,logged_now_time,number_of_classes_now,选课数量,login_time,login_place,时间差1,国家,省份,地区
0,用户44240,2020-06-17,2020-06-18,1,0,1667.28,NaN,0,1 days,1 days,1 days,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,用户44239,2020-06-17,2020-06-18,1,0,2109.75,NaN,0,1 days,1 days,1 days,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,用户44235,2020-06-17,2020-06-17,1,0,0.00,NaN,0,0 days,1 days,1 days,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,用户44237,2020-06-17,2020-06-18,1,0,10348.62,NaN,0,1 days,1 days,1 days,1,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,用户44232,2020-06-17,2020-06-18,1,0,9054.72,NaN,0,1 days,1 days,1 days,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43911,用户6,2018-09-11,2018-09-11,0,0,0.00,NaN,0,0 days,646 days,646 days,0,NaN,2018-09-11,中国广东广州,646.0,中国,广东,广州
43912,用户5,2018-09-10,2020-06-15,9,5,2116.15,NaN,0,644 days,647 days,647 days,4,NaN,2020-06-15,中国广东广州,3.0,中国,广东,广州
43913,用户4,2018-09-10,2020-06-05,4,0,341.20,NaN,0,634 days,647 days,647 days,4,NaN,2020-06-05,中国广东广州,13.0,中国,广东,广州
43914,用户3,2018-09-04,2020-06-18,2,1,370.35,NaN,0,653 days,653 days,653 days,1,NaN,2020-06-18,中国广东广州,0.0,中国,广东,广州


In [91]:
login_1

,user_id,login_time,login_place,时间差1,国家,省份,地区
1035,用户10,2018-12-29,中国广东广州,537,中国,广东,广州
1034,用户10,2018-12-27,中国广东广州,539,中国,广东,广州
1033,用户10,2018-12-26,中国广东广州,540,中国,广东,广州
1032,用户10,2018-09-27,中国广东广州,630,中国,广东,广州
1031,用户10,2018-09-18,中国广东广州,639,中国,广东,广州
...,...,...,...,...,...,...,...
34963,用户9996,2019-04-29,中国广东,416,中国,广东,NaN
34961,用户9996,2019-04-28,中国广东广州,417,中国,广东,广州
34996,用户9997,2019-04-28,中国香港,417,中国,香港,NaN
34997,用户9998,2019-04-28,中国广东广州,417,中国,广东,广州


## 2.1 用户分布分析
### 2.1.1 海内外分布

In [99]:
login.国家.value_counts()

中国    267582
英国        84
德国        22
越南        11
荷兰         8
波兰         7
南非         3
捷克         2
泰国         2
挪威         1
瑞典         1
瑞士         1
希腊         1
Name: 国家, dtype: int64

In [110]:
from pyecharts.charts import Line,Pie,Grid
from pyecharts import options as opts
line=(Line()
     .add_xaxis(login[login['国家']!='中国'].国家.value_counts().index.tolist())
     .add_yaxis('数量分布',login[login['国家']!='中国'].国家.value_counts().values.tolist())
     .set_series_opts(markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='max'),
                                                             opts.MarkPointItem(type_='min')]),
                     markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_='average')])))
pie=(Pie()
    .add('',[list(z) for z in zip(login[login['国家']!='中国'].国家.value_counts().index.tolist(),login[login['国家']!='中国'].国家.value_counts().values.tolist())],
        radius=['30%','60%'],
        center=['70%','50%'],
        rosetype='radius')
    .set_global_opts(legend_opts=opts.LegendOpts(is_show=False)))
grid=Grid()
grid.add(line,grid_opts=opts.GridOpts(pos_right='55%'))
grid.add(pie,grid_opts=opts.GridOpts(pos_left='50%'))
grid.render_notebook()

### 2.1.2中国省份分布

In [115]:
bar=(Bar(init_opts=opts.InitOpts(theme='dark'))
    .add_xaxis(login[login['国家']=='中国'].省份.value_counts().index.tolist())
    .add_yaxis('数量',login[login['国家']=='中国'].省份.value_counts().values.tolist())
    .set_global_opts(title_opts=opts.TitleOpts(title='各省份登录数量'),
                    datazoom_opts=opts.DataZoomOpts())
    .set_series_opts(markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='max'),
                                                             opts.MarkPointItem(type_='min')]),
                     markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_='average')])))
bar.render_notebook()

In [124]:
from pyecharts.charts import Map
map=(Map()
    .add('数量',[list(z) for z in zip(province,province_co)],'china')
    .set_global_opts(visualmap_opts=opts.VisualMapOpts(is_show=True,max_=81167,min_=83,is_piecewise=True),
                    title_opts=opts.TitleOpts(title='各省市分布')))
map.render_notebook()

In [123]:
pie=(Pie()
    .add('',[list(z) for z in zip(province[:15],province_co[:15])],
        radius=['30%','70%'],
        rosetype='radius')
    .set_global_opts(legend_opts=opts.LegendOpts(orient='vertical',pos_left='2%',pos_top='15%'))
    .set_series_opts(label_opts=opts.LabelOpts(formatter='{b}:{d}%',position='outer')))
pie.render_notebook()

In [122]:
province=login[login['国家']=='中国'].省份.value_counts().index.tolist()
province_co=login[login['国家']=='中国'].省份.value_counts().values.tolist()

## 用户分布行为差异
- 由于地区间差异较大，且营销策略等差异化，因此对不同省份的用户进行分析可以看出用户存在以下特点
  - 广东，澳门，贵州，重庆等地区人均学习时间较长，但澳门仅拥有16人数据。因此可以考虑该地区是否存在潜在需求
  - 山西，湖北，陕西等地则人均选课需求较高

In [125]:
users_all.groupby('省份').agg({'learn_time':['sum','mean','count'],'number_of_classes_now':['sum','mean',],'选课数量':['mean']})

learn_time                    number_of_classes_now                选课数量
            sum         mean count                   sum      mean      mean
省份                                                                          
上海    464426.72   330.552826  1405                   204  0.145196  1.250000
云南    253134.63   550.292674   460                   258  0.560870  2.142857
内蒙古   131378.54   576.221667   228                   167  0.732456       NaN
北京    326243.64   175.211407  1862                   220  0.118153  1.000000
台湾      9920.95   225.476136    44                     6  0.136364       NaN
吉林    159633.18   511.644808   312                   106  0.339744  1.000000
四川    892384.63   565.516242  1578                   789  0.500000  1.125000
天津    123309.26   306.739453   402                   106  0.263682  1.800000
宁夏    121739.74   869.569571   140                    96  0.685714  1.000000
安徽    748935.87   768.139354   975                   519  0.532308  1.000000
山东   1088362.16   633.136800  1719                   870  0.506108  1.333333
山西    560910.63   786.690926   713                   541  0.758766  3.900000
广东   9813527.51  1146.306215  8561                  5867  0.685317  1.758621
广西   1064777.01   785.233783  1356                   834  0.615044  2.000000
新疆     87401.31   556.696242   157                    76  0.484076  1.000000
江苏    888289.49   489.415697  1815                   684  0.376860  1.714286
江西    477863.60   579.228606   825                   438  0.530909  1.071429
河北   1106497.86   731.327072  1513                   906  0.598810  1.727273
河南   1396500.97   880.517636  1586                   951  0.599622  1.555556
浙江   1116008.92   630.157493  1771                   890  0.502541  1.230769
海南    132497.71   630.941476   210                   108  0.514286  1.000000
湖北   2379804.25   871.403973  2731                  2139  0.783230  3.750000
湖南    972293.12   701.510188  1386                   873  0.629870  1.111111
澳门     40371.02  2523.188750    16                     9  0.562500       NaN
甘肃    248339.38   658.725146   377                   239  0.633952  1.750000
福建    550155.44   622.347783   884                   441  0.498869  2.428571
西藏      3645.68   280.436923    13                    10  0.769231       NaN
贵州   1138613.77   964.109881  1181                   941  0.796782  1.000000
辽宁    206980.50   370.932796   558                   130  0.232975  1.000000
重庆   1109886.85   965.958964  1149                   615  0.535248  1.000000
陕西    897520.46   730.285159  1229                   550  0.447518  3.600000
青海     31433.70   523.895000    60                    25  0.416667  1.142857
香港     49369.30   783.639683    63                    31  0.492063       NaN
黑龙江   227230.35   511.780068   444                   187  0.421171  1.000000

## 2.2 用户活跃度分析

In [129]:
line=(Line()
     .add_xaxis(users_all[users_all['recently_logged']>'2020-01-01'].groupby('recently_logged').user_id.count().index.tolist())
     .add_yaxis('每日登录人数',users_all[users_all['recently_logged']>'2020-01-01'].groupby('recently_logged').user_id.count().values.tolist())
     .set_series_opts(markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='max')]))
     .set_global_opts(tooltip_opts=opts.TooltipOpts(trigger='axis')))
line.render_notebook()

### 2.2.1异常活跃点分析

In [130]:
users_all[users_all['recently_logged']=='2020-06-11'].describe()

,number_of_classes_join,number_of_classes_out,learn_time,register_logged_time,register_now_time,logged_now_time,number_of_classes_now,选课数量,时间差1
count,2488.000000,2488.000000,2488.000000,2488,2488,2488,2488.000000,12.000000,258.0
mean,1.030949,0.004823,257.791355,7 days 08:49:00.192926045,14 days 08:49:00.192926045,14 days 08:49:00.192926045,1.026125,1.500000,7.0
std,0.368992,0.074873,1692.493582,38 days 11:50:41.612168608,38 days 11:50:41.612168608,38 days 11:50:41.612168608,0.351516,1.732051,0.0
min,0.000000,0.000000,0.000000,0 days 00:00:00,7 days 00:00:00,7 days 00:00:00,0.000000,1.000000,7.0
25%,1.000000,0.000000,0.000000,0 days 00:00:00,7 days 00:00:00,7 days 00:00:00,1.000000,1.000000,7.0
50%,1.000000,0.000000,0.000000,0 days 00:00:00,7 days 00:00:00,7 days 00:00:00,1.000000,1.000000,7.0
75%,1.000000,0.000000,0.000000,0 days 00:00:00,7 days 00:00:00,7 days 00:00:00,1.000000,1.000000,7.0
max,7.000000,2.000000,58530.880000,623 days 00:00:00,630 days 00:00:00,630 days 00:00:00,7.000000,7.000000,7.0


In [131]:
users_all[users_all['recently_logged']=='2020-06-11']

,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,是否填写学校信息,register_logged_time,register_now_time,logged_now_time,number_of_classes_now,选课数量,login_time,login_place,时间差1,国家,省份,地区
17,用户43972,2020-06-11,2020-06-11,1,0,0.00,桂林电子科技大学,1,0 days,7 days,7 days,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
18,用户43973,2020-06-11,2020-06-11,1,0,0.00,桂林电子科技大学,1,0 days,7 days,7 days,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
19,用户43967,2020-06-11,2020-06-11,1,0,0.00,桂林电子科技大学,1,0 days,7 days,7 days,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
20,用户43968,2020-06-11,2020-06-11,1,0,0.00,桂林电子科技大学,1,0 days,7 days,7 days,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
21,用户43969,2020-06-11,2020-06-11,1,0,0.00,桂林电子科技大学,1,0 days,7 days,7 days,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39092,用户5260,2019-03-23,2020-06-11,1,0,2498.43,NaN,0,446 days,453 days,453 days,1,NaN,2020-06-11,中国广东佛山,7.0,中国,广东,佛山
41146,用户2839,2019-02-21,2020-06-11,2,0,3440.57,NaN,0,476 days,483 days,483 days,2,NaN,2020-06-11,中国山西长治,7.0,中国,山西,长治
43353,用户573,2018-10-31,2020-06-11,7,0,7807.80,华南师范大学,1,589 days,596 days,596 days,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43891,用户26,2018-09-30,2020-06-11,0,0,4.35,NaN,0,620 days,627 days,627 days,0,NaN,2020-06-11,中国浙江杭州,7.0,中国,浙江,杭州


In [132]:
#看看是否是省份因素
users_all[users_all['recently_logged']=='2020-06-11'].groupby('省份').user_id.count()
#每个省都有，非省份因素


省份
上海     9
云南    16
北京     6
四川    11
天津     1
宁夏     8
安徽    11
山东     7
山西     5
广东    58
广西     9
新疆     1
江苏    12
江西     4
河北    18
河南    12
浙江    20
海南     4
湖北    16
湖南     9
福建     5
贵州     2
重庆     7
陕西     3
Name: user_id, dtype: int64

In [133]:
users_all[users_all['recently_logged']=='2020-06-11'].是否填写学校信息.value_counts()
#筛选出信息差异较大，考虑是否是学校注册优惠活动

1    2292
0     196
Name: 是否填写学校信息, dtype: int64

NotImplementedError: unsupported type <class 'str'>, expected type is datetime.date

### 2.2.2工作日活跃度差异分析

In [136]:
for i in range(users_all.shape[0]):
    if i==0:
        if is_workday(pd.to_datetime(users_all.iloc[i,2])):
            users_all.loc[0,'是否工作日']=1
        else:
            users_all.loc[0,'是否工作日']=0
    else:
        if is_workday(pd.to_datetime(users_all.iloc[i,2])):
            users_all.iloc[i,-1]=1
        else:
            users_all.iloc[i,-1]=0

In [137]:
users_all.是否工作日.value_counts()

1.0    33044
0.0    10872
Name: 是否工作日, dtype: int64

In [138]:
users_all.groupby(['是否填写学校信息','是否工作日']).user_id.count()

是否填写学校信息  是否工作日
0         0.0       8964
          1.0      24383
1         0.0       1908
          1.0       8661
Name: user_id, dtype: int64

In [139]:
users_all[users_all['recently_logged']=='2020-06-11']

,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,是否填写学校信息,register_logged_time,register_now_time,logged_now_time,number_of_classes_now,选课数量,login_time,login_place,时间差1,国家,省份,地区,是否工作日
17,用户43972,2020-06-11,2020-06-11,1,0,0.00,桂林电子科技大学,1,0 days,7 days,7 days,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
18,用户43973,2020-06-11,2020-06-11,1,0,0.00,桂林电子科技大学,1,0 days,7 days,7 days,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
19,用户43967,2020-06-11,2020-06-11,1,0,0.00,桂林电子科技大学,1,0 days,7 days,7 days,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
20,用户43968,2020-06-11,2020-06-11,1,0,0.00,桂林电子科技大学,1,0 days,7 days,7 days,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
21,用户43969,2020-06-11,2020-06-11,1,0,0.00,桂林电子科技大学,1,0 days,7 days,7 days,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39092,用户5260,2019-03-23,2020-06-11,1,0,2498.43,NaN,0,446 days,453 days,453 days,1,NaN,2020-06-11,中国广东佛山,7.0,中国,广东,佛山,1.0
41146,用户2839,2019-02-21,2020-06-11,2,0,3440.57,NaN,0,476 days,483 days,483 days,2,NaN,2020-06-11,中国山西长治,7.0,中国,山西,长治,1.0
43353,用户573,2018-10-31,2020-06-11,7,0,7807.80,华南师范大学,1,589 days,596 days,596 days,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
43891,用户26,2018-09-30,2020-06-11,0,0,4.35,NaN,0,620 days,627 days,627 days,0,NaN,2020-06-11,中国浙江杭州,7.0,中国,浙江,杭州,1.0


## 2.3用户流失情况分析

In [144]:
ar=[str(aa)[:-14] for aa in users_all.groupby('logged_now_time').logged_now_time.count().index.tolist()]
kr=users_all.groupby('logged_now_time').logged_now_time.count().values.tolist()

In [146]:
l1=(Line()
   .add_xaxis(ar[:10])
   .add_yaxis('',kr[:10])
   .set_series_opts(markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_='average')])))
l2=(Line()
   .add_xaxis(ar[:30])
   .add_yaxis('',kr[:30])
   .set_series_opts(markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_='average')])))
l3=(Line()
   .add_xaxis(ar[:150])
   .add_yaxis('',kr[:150])
   .set_series_opts(markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_='average')])))
l4=(Line()
   .add_xaxis(ar[:])
   .add_yaxis('',kr[:])
   .set_series_opts(markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_='average')])))
grid=Grid()
grid.add(l1,grid_opts=opts.GridOpts(pos_left='55%',pos_top='55%'))
grid.add(l2,grid_opts=opts.GridOpts(pos_right='55%',pos_top='55%'))
grid.add(l3,grid_opts=opts.GridOpts(pos_left='55%',pos_bottom='55%'))
grid.add(l4,grid_opts=opts.GridOpts(pos_right='55%',pos_bottom='55%'))
grid.render_notebook()

### 2.3.1分省份对流失情况分析

In [147]:
users_all.groupby(['省份','logged_now_time']).user_id.count().unstack()

logged_now_time,2 days,4 days,7 days,8 days,9 days,11 days,12 days,14 days,15 days,19 days,...,647 days,653 days,562 days,571 days,612 days,544 days,559 days,613 days,615 days,566 days
省份,,,,,,,,,,,,,,,,,,,,,
上海,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
云南,1.0,1.0,NaN,NaN,2.0,1.0,2.0,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
内蒙古,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
北京,NaN,NaN,3.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
台湾,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
吉林,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
四川,1.0,1.0,1.0,1.0,NaN,1.0,2.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
天津,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
宁夏,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [158]:
for i in range(users_all.shape[0]):
    if int(str(users_all.loc[i,'logged_now_time'])[:-14]) > 150:        
        users_all.loc[i,'流失时间划分']='大于150天'
    elif 90 <= int(str(users_all.loc[i,'logged_now_time'])[:-14]) < 150:        
        users_all.loc[i,'流失时间划分']='大于90天'
    elif 30 <= int(str(users_all.loc[i,'logged_now_time'])[:-14]) < 90:        
        users_all.loc[i,'流失时间划分']='大于30天'
    elif 15 <= int(str(users_all.loc[i,'logged_now_time'])[:-14]) < 30:        
        users_all.loc[i,'流失时间划分']='大于15天'
    elif 7 <= int(str(users_all.loc[i,'logged_now_time'])[:-14]) < 15:        
        users_all.loc[i,'流失时间划分']='大于7天'
    elif 0 <= int(str(users_all.loc[i,'logged_now_time'])[:-14]) < 7:        
        users_all.loc[i,'流失时间划分']='7天内'
users_all

,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,是否填写学校信息,register_logged_time,register_now_time,...,number_of_classes_now,选课数量,login_time,login_place,时间差1,国家,省份,地区,是否工作日,流失时间划分
0,用户44240,2020-06-17,2020-06-18,1,0,1667.28,NaN,0,1 days,1 days,...,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,7天内
1,用户44239,2020-06-17,2020-06-18,1,0,2109.75,NaN,0,1 days,1 days,...,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,7天内
2,用户44235,2020-06-17,2020-06-17,1,0,0.00,NaN,0,0 days,1 days,...,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,7天内
3,用户44237,2020-06-17,2020-06-18,1,0,10348.62,NaN,0,1 days,1 days,...,1,4.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,7天内
4,用户44232,2020-06-17,2020-06-18,1,0,9054.72,NaN,0,1 days,1 days,...,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,7天内
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43911,用户6,2018-09-11,2018-09-11,0,0,0.00,NaN,0,0 days,646 days,...,0,NaN,2018-09-11,中国广东广州,646.0,中国,广东,广州,1.0,大于150天
43912,用户5,2018-09-10,2020-06-15,9,5,2116.15,NaN,0,644 days,647 days,...,4,NaN,2020-06-15,中国广东广州,3.0,中国,广东,广州,1.0,大于150天
43913,用户4,2018-09-10,2020-06-05,4,0,341.20,NaN,0,634 days,647 days,...,4,NaN,2020-06-05,中国广东广州,13.0,中国,广东,广州,1.0,大于150天
43914,用户3,2018-09-04,2020-06-18,2,1,370.35,NaN,0,653 days,653 days,...,1,NaN,2020-06-18,中国广东广州,0.0,中国,广东,广州,1.0,大于150天


In [159]:
users_all.groupby(['省份','流失时间划分']).user_id.count().unstack()

流失时间划分,7天内,大于150天,大于15天,大于30天,大于7天,大于90天
省份,,,,,,
上海,4.0,1160.0,23.0,93.0,6.0,119.0
云南,7.0,191.0,8.0,154.0,6.0,94.0
内蒙古,NaN,74.0,6.0,30.0,1.0,117.0
北京,5.0,1580.0,8.0,118.0,6.0,145.0
台湾,NaN,33.0,NaN,7.0,1.0,3.0
吉林,3.0,182.0,3.0,31.0,2.0,91.0
四川,8.0,886.0,14.0,204.0,9.0,456.0
天津,5.0,296.0,3.0,31.0,1.0,66.0
宁夏,3.0,39.0,19.0,20.0,1.0,58.0


In [161]:
users_all.groupby(['流失时间划分']).agg(['mean']).T


,流失时间划分,7天内,大于150天,大于15天,大于30天,大于7天,大于90天
number_of_classes_join,mean,0.480176,0.169367,0.648318,0.937695,0.934313,1.107405
number_of_classes_out,mean,0.022026,0.009889,0.095821,0.017529,0.002448,0.042468
learn_time,mean,1187.201894,410.942712,1333.577819,829.458077,43.925129,1126.302288
是否填写学校信息,mean,0.030837,0.032917,0.206932,0.176328,0.906161,0.495097
register_logged_time,mean,1 days 19:08:11.629955947,43 days 13:56:25.728213937,8 days 00:29:21.467889908,16 days 15:25:55.709822978,0 days 02:17:28.714810281,35 days 19:51:42.988186240
register_now_time,mean,3 days 21:08:43.348017621,371 days 18:13:09.228840708,21 days 11:41:39.082568807,61 days 00:50:28.948281847,7 days 07:20:02.937576499,114 days 17:26:23.599722030
logged_now_time,mean,3 days 21:08:43.348017621,371 days 18:13:09.228840708,21 days 11:41:39.082568807,61 days 00:50:28.948281847,7 days 07:20:02.937576499,114 days 17:26:23.599722030
number_of_classes_now,mean,0.45815,0.159478,0.552497,0.920167,0.931865,1.064937
选课数量,mean,1.797357,NaN,NaN,NaN,1.642857,NaN
时间差1,mean,2.086538,320.655484,13.310023,42.457285,9.315315,73.506489


# 3 用户课程选择分析
## 3.1 选课人数最多的课程

In [163]:
bar=(Bar()
    .add_xaxis(stu_info.course_id.value_counts().index.tolist()[:30])
    .add_yaxis('',stu_info.course_id.value_counts().values.tolist())
    .set_global_opts(datazoom_opts=opts.DataZoomOpts())
    .set_series_opts(markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='max'),
                                                            opts.MarkPointItem(type_='min')])))
bar.render_notebook()

### 3.1.1最受欢迎的免费课程

In [166]:
bar=(Bar(init_opts=opts.InitOpts(theme='intographic'))
    .add_xaxis(stu_info[stu_info['price']==0].course_id.value_counts().index.tolist()[:30])
    .add_yaxis('',stu_info[stu_info['price']==0].course_id.value_counts().values.tolist()[:30])
    .set_global_opts(datazoom_opts=opts.DataZoomOpts())
    .set_series_opts(markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='max'),
                                                            opts.MarkPointItem(type_='min')]),
                    markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_='average')])))
bar.render_notebook()

### 3.1.2 最后欢迎的收费课程

In [168]:
bar=(Bar(init_opts=opts.InitOpts(theme='intographic'))
    .add_xaxis(stu_info[stu_info['price']!=0].course_id.value_counts().index.tolist()[:30])
    .add_yaxis('',stu_info[stu_info['price']!=0].course_id.value_counts().values.tolist()[:30])
    .set_global_opts(datazoom_opts=opts.DataZoomOpts())
    .set_series_opts(markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='max'),
                                                            opts.MarkPointItem(type_='min')]),
                    markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_='average')])))
bar.render_notebook()

In [175]:
stu_info.groupby('price').agg({'learn_process':['sum','mean'],'user_id':['count']})
# 可以看出课程的价格为109时候，用户学习时间和数量都相对较优，但是随着价格的上升，用户学习时间突然下降

learn_process            user_id
                 sum       mean   count
price                                  
0.0           987006  13.884284   71088
29.0            1460   8.156425     179
49.0            7310  39.728261     184
59.0            1005   5.345745     188
79.0           21325  20.663760    1032
99.0           19474  12.871117    1513
109.0         431260  37.989782   11352
129.0          57465   2.904180   19787
159.0          11302  15.653740     722
169.0          10379  64.067901     162
179.0          24735   8.101867    3053
199.0         479444  14.919218   32136
219.0           2540  10.495868     242
229.0             72   9.000000       8
299.0         461420  15.545972   29681
319.0          33768   6.971098    4844
369.0          33872  44.046814     769
399.0           5080   8.552189     594
499.0          83560  10.835062    7712
600.0          37750  44.940476     840
700.0          28433  84.622024     336
800.0           2000  71.428571      28
899.0          40352  35.678161    1131
999.0              0   0.000000     222
1299.0          6050  55.000000     110
3000.0          5670   2.008502    2823